In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
#import torchtext
import io
from happytransformer import HappyTextClassification 


In [2]:
!nvidia-smi
print(torch.cuda.is_available())

Sun Jan 15 01:34:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.77       Driver Version: 512.77       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 18%   42C    P0    19W / 215W |    720MiB /  8192MiB |     37%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
happy_tc = HappyTextClassification("DISTILBERT", "distilbert-base-uncased-finetuned-sst-2-english")

01/15/2023 01:35:03 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [4]:
happy_tc = HappyTextClassification("DISTILBERT", "distilbert-base-uncased",num_labels=3)
from happytransformer import TCTrainArgs
args = TCTrainArgs(batch_size=8)
happy_tc.train("train.csv", args=args)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

  0%|          | 0/9489 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.4645, 'learning_rate': 4.73653704289177e-05, 'epoch': 0.16}
{'loss': 0.3689, 'learning_rate': 4.4730740857835393e-05, 'epoch': 0.32}
{'loss': 0.3449, 'learning_rate': 4.2096111286753086e-05, 'epoch': 0.47}
{'loss': 0.3302, 'learning_rate': 3.946148171567078e-05, 'epoch': 0.63}
{'loss': 0.3344, 'learning_rate': 3.682685214458848e-05, 'epoch': 0.79}
{'loss': 0.2918, 'learning_rate': 3.419222257350617e-05, 'epoch': 0.95}
{'loss': 0.2544, 'learning_rate': 3.155759300242386e-05, 'epoch': 1.11}
{'loss': 0.2366, 'learning_rate': 2.8922963431341553e-05, 'epoch': 1.26}
{'loss': 0.2415, 'learning_rate': 2.6288333860259252e-05, 'epoch': 1.42}
{'loss': 0.232, 'learning_rate': 2.365370428917694e-05, 'epoch': 1.58}
{'loss': 0.2398, 'learning_rate': 2.1019074718094636e-05, 'epoch': 1.74}
{'loss': 0.2375, 'learning_rate': 1.838444514701233e-05, 'epoch': 1.9}
{'loss': 0.207, 'learning_rate': 1.5749815575930024e-05, 'epoch': 2.06}
{'loss': 0.1591, 'learning_rate': 1.3115186004847718e-05, 'epo



Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 2445.3918, 'train_samples_per_second': 31.034, 'train_steps_per_second': 3.88, 'train_loss': 0.2473712116715428, 'epoch': 3.0}


In [7]:
print(happy_tc.classify_text("I am so happy about this!"))
print(happy_tc.classify_text("This is the most dissapointing news I have had in a while."))
print(happy_tc.classify_text("This house was built three years ago."))
print(happy_tc.classify_text("I am amazed by how amazing these nuts are!"))
print(happy_tc.classify_text("They have no milk, this sucks."))

TextClassificationResult(label='LABEL_2', score=0.9997194409370422)
TextClassificationResult(label='LABEL_2', score=0.9994916915893555)
TextClassificationResult(label='LABEL_2', score=0.9978398084640503)
TextClassificationResult(label='LABEL_2', score=0.9997971653938293)
TextClassificationResult(label='LABEL_0', score=0.9892792105674744)


In [6]:
import evaluate
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

In [14]:
#model = fasttext.train_unsupervised('data/enwik9')
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data



In [9]:
class sentiment_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_class):
        super(TweetRNN, self).__init__()
        self.emb = nn.Embedding.from_pretrained(glove.vectors) # Replace with embeddings
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_class)

    def forward(self, x): 
        # Look-up the embeddings 
        x = self.emb(x) 
        # Set the initial hidden states 
        h0 = torch.zeros(1, x.size(0), self.hidden_size) 
        c0 = torch.zeros(1, x.size(0), self.hidden_size) 
        # Forward propagate the RNN 
        out, __ = self.rnn(x, (h0, c0)) 
        # Pass the output of the last step to the classifier return 
        self.fc(out[:,-1,:])

